<a href="https://colab.research.google.com/github/s-a-malik/zero-shot/blob/main/few_shot_loading_data_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import torch.utils.data as data
import json
import nltk
import os
from PIL import Image
from transformers import BertTokenizer, BertModel
from torchvision.transforms import Compose

class DefaultTransform(Compose):

  def __init__(self, image_size: int):


class Zanim(data.Dataset):

  def __init__(self, root: str, json_path: str, image_transforms: Compose = None):
    """
    :param root: the path to the root directory of the dataset
    :param json_path: the path to the json file containing the annotations
    :param transforms: a composed set of transforms
    """
    if not(root in json_path):
        json_path = os.path.join(root, json_path)
    self.root = root
    self.transforms = image_transforms
    with open(json_path) as annotations:
      annotations = json.load(annotations)

    self.tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
    self.bert_model = BertModel.from_pretrained("bert-base-uncased")

    self.ids = [i['id'] for i in annotations['images']]
    self.image_files = [i['file_name'] for i in annotations['images']]
    self.species_id = [annotations['annotations'][i['id']]['category_id'] for i in annotations['images']]
    self.species_name = [annotations['categories'][i]['name'] for i in self.species_id]
    self.descriptions = [annotations['categories'][index]['description'] for index in self.species_id]

  def __len__(self):
    return len(self.ids)

  def __getitem__(self, index):
    image = Image.open(os.path.join(self.root, self.image_files[index]))
    if self.transforms:
      image = self.transforms(image)

    description = self.descriptions[index] 
    species_id = self.species_id[index]
    tokens = self.tokenizer(description, return_tensors="pt")
    embedding = self.bert_model(**tokens)
    return (image, embedding), species_id, {'description': description, 
                                            'species': self.species_name[index]}


In [5]:
z = Zanim(root="/content/drive/My Drive/NLP project/Dataset", json_path="train.json", image_transforms=None)

In [ ]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
bert_model = BertModel.from_pretrained("bert-base-uncased")
z.descriptions

In [29]:
!pip install transformers

     |████████████████████████████████| 2.0MB 5.2MB/s 
     |████████████████████████████████| 3.2MB 17.1MB/s 
     |████████████████████████████████| 890kB 35.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=6ddbc184e8b993cd60f8408c61c3ccacb6e81080684223d2d1bb3221df73d372
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [24]:
print("Species names")
print("\n".join(z.species_name[:10]))
print("\nSpecies ID")
print(z.species_id[:10])
print("\nDescriptions")
print("\n".join(z.descriptions[:10]))
print("\nImage files examples")
print("\n".join(z.image_files[:10]))

Species names
Marmota flaviventris
Dipsosaurus dorsalis
Gopherus agassizii
Grampus griseus
Sterna paradisaea
Fratercula arctica
Cistothorus palustris
Falco mexicanus
Crocodylus acutus
Lanius ludovicianus

Species ID
[524, 607, 665, 388, 100, 84, 242, 138, 545, 205]

Descriptions
Yellow-bellied marmots are rodents similar to squirrels, but bigger. They have grizzled brownish fur and a yellow belly, with a whitish spot between their eyes. Their ears are small and round and they have a short white muzzle with a black nose. Their bodies are heavy-set and they have short legs and a reddish-brown furry tail.
The Desert iguana is one of the most common lizards of the Sonoran and Mojave deserts of the southwestern United States and northwestern Mexico. It is pale gray-tan to cream in color with a light brown reticulated pattern on its back and sides. Down the center of the back is a row of slightly-enlarged, keeled dorsal scales that become slightly larger as you move down the back. The reticu

# Generating SSH key for using drive and github

In [1]:
from google.colab import drive 
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**PUT YOUR EMAIL BELOW**

In [ ]:
!ssh-keygen -t ed25519 -C "youmed.tech@gmail.com"

Generating public/private ed25519 key pair.
Enter file in which to save the key (/root/.ssh/id_ed25519): /content/drive/My Drive/ssh/id_ed25519
Enter passphrase (empty for no passphrase): 
Enter same passphrase again: 
Your identification has been saved in /content/drive/My Drive/ssh/id_ed25519.
Your public key has been saved in /content/drive/My Drive/ssh/id_ed25519.pub.
The key fingerprint is:
SHA256:gj/J3Z3Uwyr2H/bwCbhphn+OCtJ3aXon4hoj9MskOUI youmed.tech@gmail.com
The key's randomart image is:
+--[ED25519 256]--+
|                 |
|                 |
|                 |
|     .       o   |
|   E... S   . +  |
|  . .o++ . ooo . |
|   . *=B..==+.+  |
|    . B.*++B++.=.|
|       +o+B=*o..o|
+----[SHA256]-----+


**CREATE A FOLDER SSH IN YOUR GDRIVE**

In [ ]:
!rm -rf /etc/ssh/ssh_config
!touch /etc/ssh/ssh_config
!echo "Host *" >> /etc/ssh/ssh_config
!echo " AddKeysToAgent yes" >> /etc/ssh/ssh_config
!echo " IgnoreUnknown UseKeychain" >> /etc/ssh/ssh_config
!echo " IdentityFile \"/content/drive/My Drive/ssh/id_ed25519\"" >> /etc/ssh/ssh_config

In [ ]:
!eval "$(ssh-agent -s)" && ssh-add -k /content/drive/My\ Drive/ssh/id_ed25519

Agent pid 686
Identity added: /content/drive/My Drive/ssh/id_ed25519 (youmed.tech@gmail.com)


**COPY THE KEY AND ADD TO YOUR GITHUB**

In [ ]:
!ssh-keyscan -t rsa github.com >> ~/.ssh/known_hosts
!git clone git@github.com:s-a-malik/zero-shot.git

# github.com:22 SSH-2.0-babeld-8514a139
Cloning into 'zero-shot'...
remote: Enumerating objects: 26, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 26 (delta 7), reused 14 (delta 2), pack-reused 0
Receiving objects: 100% (26/26), 679.75 KiB | 2.57 MiB/s, done.
Resolving deltas: 100% (7/7), done.
